In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import keras
from keras.utils.np_utils import to_categorical 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import cv2
import os



A:\anaconda\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


# Loading Data set from AVR_data folder

In [33]:
list_rgb_img=[]
labels=[]
preprocessed_faces=np.zeros((859,96,96,3))
dir="AVR_data\\"
classes_path=[os.path.join(dir,i) for i in sorted(os.listdir(dir))]
images_path=[]
for j in classes_path:
    images_path.append([os.path.join(j,i) for i in sorted(os.listdir(j))])
for i in images_path:
    for j in i:
        image=cv2.imread(j)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        list_rgb_img.append(image)
        labels.append(j.split("\\")[2])
le=LabelEncoder()
le.fit(labels)
labels=le.transform(labels)



# Face detection and reshaping to 96X96 

In [25]:
TEMPLATE = np.float32([
    (0.0792396913815, 0.339223741112), (0.0829219487236, 0.456955367943),
    (0.0967927109165, 0.575648016728), (0.122141515615, 0.691921601066),
    (0.168687863544, 0.800341263616), (0.239789390707, 0.895732504778),
    (0.325662452515, 0.977068762493), (0.422318282013, 1.04329000149),
    (0.531777802068, 1.06080371126), (0.641296298053, 1.03981924107),
    (0.738105872266, 0.972268833998), (0.824444363295, 0.889624082279),
    (0.894792677532, 0.792494155836), (0.939395486253, 0.681546643421),
    (0.96111933829, 0.562238253072), (0.970579841181, 0.441758925744),
    (0.971193274221, 0.322118743967), (0.163846223133, 0.249151738053),
    (0.21780354657, 0.204255863861), (0.291299351124, 0.192367318323),
    (0.367460241458, 0.203582210627), (0.4392945113, 0.233135599851),
    (0.586445962425, 0.228141644834), (0.660152671635, 0.195923841854),
    (0.737466449096, 0.182360984545), (0.813236546239, 0.192828009114),
    (0.8707571886, 0.235293377042), (0.51534533827, 0.31863546193),
    (0.516221448289, 0.396200446263), (0.517118861835, 0.473797687758),
    (0.51816430343, 0.553157797772), (0.433701156035, 0.604054457668),
    (0.475501237769, 0.62076344024), (0.520712933176, 0.634268222208),
    (0.565874114041, 0.618796581487), (0.607054002672, 0.60157671656),
    (0.252418718401, 0.331052263829), (0.298663015648, 0.302646354002),
    (0.355749724218, 0.303020650651), (0.403718978315, 0.33867711083),
    (0.352507175597, 0.349987615384), (0.296791759886, 0.350478978225),
    (0.631326076346, 0.334136672344), (0.679073381078, 0.29645404267),
    (0.73597236153, 0.294721285802), (0.782865376271, 0.321305281656),
    (0.740312274764, 0.341849376713), (0.68499850091, 0.343734332172),
    (0.353167761422, 0.746189164237), (0.414587777921, 0.719053835073),
    (0.477677654595, 0.706835892494), (0.522732900812, 0.717092275768),
    (0.569832064287, 0.705414478982), (0.635195811927, 0.71565572516),
    (0.69951672331, 0.739419187253), (0.639447159575, 0.805236879972),
    (0.576410514055, 0.835436670169), (0.525398405766, 0.841706377792),
    (0.47641545769, 0.837505914975), (0.41379548902, 0.810045601727),
    (0.380084785646, 0.749979603086), (0.477955996282, 0.74513234612),
    (0.523389793327, 0.748924302636), (0.571057789237, 0.74332894691),
    (0.672409137852, 0.744177032192), (0.572539621444, 0.776609286626),
    (0.5240106503, 0.783370783245), (0.477561227414, 0.778476346951)])

TPL_MIN, TPL_MAX = np.min(TEMPLATE, axis=0), np.max(TEMPLATE, axis=0)
MINMAX_TEMPLATE = (TEMPLATE - TPL_MIN) / (TPL_MAX - TPL_MIN)

In [28]:
#face detector
import dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.DAT')
OUTER_EYES_AND_NOSE = [36, 45, 33]

def getAllFaceBoundingBoxes(rgbImg):

        assert rgbImg is not None

        try:
            return detector(rgbImg, 1)
        except Exception as e:
            print("Warning: {}".format(e))
            # In rare cases, exceptions are thrown.
            return []
def getLargestFaceBoundingBox(rgbImg, skipMulti=False):

        assert rgbImg is not None

        faces = getAllFaceBoundingBoxes(rgbImg)
        if (not skipMulti and len(faces) > 0) or len(faces) == 1:
            return max(faces, key=lambda rect: rect.width() * rect.height())
        else:
            return None
def findLandmarks(rgbImg, bb):

        assert rgbImg is not None
        assert bb is not None

        points = predictor(rgbImg, bb)
        return list(map(lambda p: (p.x, p.y), points.parts()))
count=0
labels1=[]
for i in range(len(list_rgb_img)):
    rgbImg=list_rgb_img[i]
    bb=getLargestFaceBoundingBox(rgbImg)
    if(bb is None):
        print("face not detected")
    if(bb is not None):
        landmarks = findLandmarks(rgbImg, bb)
        labels1.append(labels[i])        
        npLandmarks = np.float32(landmarks)
        npLandmarkIndices = np.array(OUTER_EYES_AND_NOSE)
        H = cv2.getAffineTransform(npLandmarks[npLandmarkIndices],
                                   96 * MINMAX_TEMPLATE[npLandmarkIndices])
        preprocessed_faces[count] = cv2.warpAffine(rgbImg, H, (96, 96))
        count+=1

In [ ]:
preprocessed_faces=preprocessed_faces[0:757]
preprocessed_faces=preprocessed_faces/255
labels=np.asarray(labels1)

In [ ]:
(trainimages, testimages, trainlabels, testlabels) = train_test_split(
preprocessed_faces, labels, test_size=0.15, random_state=42)

# Face Recognition

In [ ]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=42):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


model = resnet_v1(input_shape=(96,96,3), depth=20)
# model.summary()

In [49]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
model.fit(trainimages,trainlabels,epochs=20,batch_size=128,validation_split=0.2,shuffle=True)

Train on 512 samples, validate on 129 samples
Epoch 1/20
512/512 [==============================] - 6s 12ms/step - loss: 0.4066 - acc: 0.9238 - val_loss: 2.7607 - val_acc: 0.4341
Epoch 2/20
512/512 [==============================] - 1s 2ms/step - loss: 0.5349 - acc: 0.9180 - val_loss: 10.9916 - val_acc: 0.0930
Epoch 3/20
512/512 [==============================] - 1s 2ms/step - loss: 0.3072 - acc: 0.9668 - val_loss: 1.0121 - val_acc: 0.8062
Epoch 4/20
512/512 [==============================] - 1s 2ms/step - loss: 0.1773 - acc: 1.0000 - val_loss: 0.9318 - val_acc: 0.8372
Epoch 5/20
512/512 [==============================] - 1s 2ms/step - loss: 0.1731 - acc: 1.0000 - val_loss: 0.9355 - val_acc: 0.8527
Epoch 6/20
512/512 [==============================] - 1s 2ms/step - loss: 0.1702 - acc: 1.0000 - val_loss: 0.9196 - val_acc: 0.8527
Epoch 7/20
512/512 [==============================] - 1s 2ms/step - loss: 0.1683 - acc: 1.0000 - val_loss: 0.9088 - val_acc: 0.8372
Epoch 8/20
512/512 [========

In [ ]:
img_pairs=int(testimages.shape[0]*(testimages.shape[0]-1)/2)
img=np.zeros((img_pairs,1,2))
label_result=np.zeros((img_pairs))
count=0
for  i in range(testimages.shape[0]):
    for j in range(testimages.shape[0]):
        if(i!=j and count<img_pairs):
#             print(i)
            img[count,0,0]=i
            img[count,0,1]=j
            if(testlabels[i]==testlabels[j]):
                label_result[count]=1
            else: 
                label_result[count]=0
            count+=1

    

In [ ]:
predicted_result=np.zeros((img_pairs))
first_image=np.zeros((1,96,96,3))
second_image=np.zeros((1,96,96,3))
for i in range(img_pairs):
    first_image[0]=testimages[int(img[i,0,0])]
    second_image[0]=testimages[int(img[i,0,1])]
    print(i)
    x=model.predict(first_image)
    y=model.predict(second_image)
    if(np.argmax(x)==np.argmax(y)):
        predicted_result[i]=1
    else:
        predicted_result[i]=0
        
        

In [48]:
print("test_images accuracy:",accuracy_score(label_result,predicted_result)*100)

test_images accuracy: 98.72690575997515
